# Dash Anomalías de temperatura

La apliclación desplegada tiene como objetivo de visualización los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. El archivo presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990.

En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra.



In [2]:
#Se cargan las libreias necesarias
import pandas as pd
from time import mktime
from datetime import datetime, timedelta
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Temas para las figuras
sns.set_theme()
sns.set_context("paper")

## Manejo de la base de datos

In [5]:
# Leer los datos

df_temperature = pd.read_csv('./data/originaldata_temp.csv')
df_temperature.head()

,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean,Unnamed: 14
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418,NaN
1,1850,53.000,47.000,44.000,48.000,49.000,49.000,52.000,55.000,57.000,55.000,58.000,58.000,NaN,NaN
2,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233,NaN
3,1851,58.000,55.000,56.000,55.000,54.000,53.000,54.000,59.000,47.000,51.000,48.000,48.000,NaN,NaN
4,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229,NaN


In [6]:
# Eliminar columna vacía

df_temperature = df_temperature.drop(df_temperature.columns[df_temperature.columns.str.contains('Unnamed')], axis=1)
df_temperature.head()

,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418
1,1850,53.000,47.000,44.000,48.000,49.000,49.000,52.000,55.000,57.000,55.000,58.000,58.000,NaN
2,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233
3,1851,58.000,55.000,56.000,55.000,54.000,53.000,54.000,59.000,47.000,51.000,48.000,48.000,NaN
4,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229


Con este código logramos eliminar la columna que no representar información sobre los datos que se desean analizar

In [7]:
# Eliminar filas intermedias
df_temperature = df_temperature.drop(df_temperature.index[1::2]).reset_index(drop=True)
df_temperature.head()


,year,january,february,march,april,may,june,july,august,september,october,november,december,yearmean
0,1850,-0.675,-0.333,-0.591,-0.589,-0.508,-0.344,-0.160,-0.208,-0.385,-0.533,-0.283,-0.404,-0.418
1,1851,-0.201,-0.469,-0.646,-0.542,-0.198,-0.137,-0.097,-0.102,-0.091,-0.008,-0.082,-0.228,-0.233
2,1852,-0.375,-0.477,-0.560,-0.585,-0.127,-0.084,0.005,-0.136,-0.002,-0.172,-0.305,0.065,-0.229
3,1853,-0.233,-0.404,-0.280,-0.386,-0.268,-0.142,-0.083,-0.057,-0.250,-0.392,-0.411,-0.337,-0.270
4,1854,-0.381,-0.361,-0.243,-0.334,-0.291,-0.299,-0.179,-0.239,-0.217,-0.095,-0.410,-0.450,-0.292


Este código se ejecutó para eliminar las filas que no son parte del estudio, con el objetivo de quedarnos con aquellas que aportan al desarrollo de este.

In [11]:
# Cambiar el formato del dataframe para que pueda tener variables identificadores de mi elección. 
## vamos a cambiar las columnas que son valores de meses, a elementos de la columna month para cada año.

# Convertir a formato de serie de tiempo

# Crear las nuevas columnas "month", "year" y "value"
df_timeseries = pd.DataFrame(columns=['year', 'month', 'value'])

# Iterar sobre las filas del DataFrame original
for index, row in df_temperature.iterrows():
    year = row['year']
    for month in df_temperature.columns[1:-1]:  # Ignorar la primera columna (year) y la última (yearmean)
        new_row = pd.DataFrame({'year': [year], 'month': [month], 'value': [row[month]]})
        df_timeseries = pd.concat([df_timeseries, new_row], ignore_index=True)


# Asegurarse de que los tipos de datos sean correctos
df_timeseries['year'] = df_timeseries['year'].astype(int)

# Ver el nuevo DataFrame transformado
df_timeseries.head(15)


      year      month  value
0     1850    january -0.675
1     1850   february -0.333
2     1850      march -0.591
3     1850      april -0.589
4     1850        may -0.508
...    ...        ...    ...
2083  2023     august  1.199
2084  2023  september  1.347
2085  2023    october -9.999
2086  2023   november -9.999
2087  2023   december -9.999

[2088 rows x 3 columns]


,year,month,value
0,1850,january,-0.675
1,1850,february,-0.333
2,1850,march,-0.591
3,1850,april,-0.589
4,1850,may,-0.508
5,1850,june,-0.344
6,1850,july,-0.160
7,1850,august,-0.208
8,1850,september,-0.385
9,1850,october,-0.533


Este código usa pandas para transformar un DataFrame llamado 'df_temperature' en una serie de tiempo representada por otro DataFrame llamado 'df_timeseries'. Se crea un nuevo DataFrame con columnas 'year', 'month' y 'value'. Luego, se retoma sobre las filas del DataFrame original para extraer el año y cada valor mensual, creando un nuevo registro por cada mes con su respectivo dato. Después, se asegura que el tipo de dato de la columna 'year' sea entero y se muestra una vista previa de las primeras 15 filas del nuevo DataFrame 'df_timeseries' que contiene la serie de tiempo transformada.

In [13]:
# Crear la nueva columna de fecha en formato %Y-%m
df_timeseries['date'] = pd.to_datetime(df_timeseries['year'].astype(str) + '-' + df_timeseries['month'], format='%Y-%B')

# Formatear la fecha para que solo muestre mes y año
df_timeseries['date'] = df_timeseries['date'].dt.strftime('%Y-%m')

# Ver el DataFrame final con la nueva columna de fecha
print(df_timeseries)


      year      month  value     date
0     1850    january -0.675  1850-01
1     1850   february -0.333  1850-02
2     1850      march -0.591  1850-03
3     1850      april -0.589  1850-04
4     1850        may -0.508  1850-05
...    ...        ...    ...      ...
2083  2023     august  1.199  2023-08
2084  2023  september  1.347  2023-09
2085  2023    october -9.999  2023-10
2086  2023   november -9.999  2023-11
2087  2023   december -9.999  2023-12

[2088 rows x 4 columns]


Este código transforma el formato de fecha que se tiene de los datos originales con el fin de tener los datos en el formato requerido para la serie de tiempo.

In [14]:
# Eliminar las últimas tres filas
df_timeseries = df_timeseries.iloc[:-3]

# Ver el DataFrame después de eliminar las filas
print(df_timeseries)



      year      month  value     date
0     1850    january -0.675  1850-01
1     1850   february -0.333  1850-02
2     1850      march -0.591  1850-03
3     1850      april -0.589  1850-04
4     1850        may -0.508  1850-05
...    ...        ...    ...      ...
2080  2023        may  0.871  2023-05
2081  2023       june  1.052  2023-06
2082  2023       july  1.150  2023-07
2083  2023     august  1.199  2023-08
2084  2023  september  1.347  2023-09

[2085 rows x 4 columns]


In [15]:
df_timeseries.to_csv("df_timeseries.csv", index=False)

# Construcción del dash

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)
df_tseries.head()


# Definimos la función encargada de calcular la media móvil y el intervalo de 
## En este código '1.96' es el valor z para un intervalo de confianza del 95%.

# Función para calcular la media móvil y el intervalo de confianza
def calculate_rolling_mean(df, window_size=12, num_of_std=1.96):
    df['rolling_mean'] = df['value'].rolling(window=window_size).mean()
    df['rolling_std'] = df['value'].rolling(window=window_size).std()
    df['upper_band'] = df['rolling_mean'] + (df['rolling_std'] * num_of_std)
    df['lower_band'] = df['rolling_mean'] - (df['rolling_std'] * num_of_std)
    return df

# Orden de los meses
month_order = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
# Pivotar los datos para tener años en el eje x, meses en el eje y
heatmap_data = df_tseries.pivot(index='month', columns='year', values='value')
# Configurar el orden de los meses en el eje y
heatmap_data = heatmap_data.reindex(month_order)


# Convertir la columna 'date' a tipo datetime si aún no lo está
df_tseries['date'] = pd.to_datetime(df_tseries['date'])
# Agregar una columna 'decade' para agrupar por década
df_tseries['decade'] = (df_tseries['year'] // 10) * 10
# Agregar una columna 'color' basada en la condición de la anomalía
df_tseries['color'] = df_tseries['value'].apply(lambda x: 'red' if x > 0 else 'blue')

df_tseries['year'] = df_tseries['date'].dt.year
df_tseries['month'] = df_tseries['date'].dt.month
# Encuentra los tres años más cálidos, excluyendo el año actual
current_year = datetime.now().year
top_years = df_tseries[df_tseries['year'] != current_year].groupby('year')['value'].mean().nlargest(3).index
# Encuentra los valores máximos para cada mes
max_values = df_tseries.groupby('month')['value'].max()

app = dash.Dash(__name__, suppress_callback_exceptions=True)




app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Main(style={'display':'grid', 'height':'100%','minHeight':'100vh','gridTemplateColumns': '50vh 1fr', 'gridTemplateRows': '50px 1fr 30px', }, children=[
       
        html.Nav(className='nav', style={'display':'fixed','gridArea':'nav', 'gridColumnStart':'1','gridColumnEnd':'2','gridRowStart':'1','gridRowEnd':'5', 'backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}, children=[
            html.H1('Anomalia de Temperatura de Superficie Global', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1.5rem', 'textAlign':'center'}),
            html.H2('¿Cuáles han sido las variaciones más significativas de la temperatura de la superficie global a lo largo del tiempo?', style={'color': 'black', 'marginBottom': '20px', 'fontSize':'1rem', 'textAlign':'left'}),
            html.H3('¿Aumentará la temperatura los próximos meses?', style={'color': '#de3100', 'marginBottom': '20px', 'fontSize':'1rem','fontWeight':'900', 'textAlign':'left'}),
            html.Ul(style={'display': 'flex', 'justifyContent': 'center','flexDirection':'column', 'alignItems':'center', 'width':'100%','margin':'10px 0','padding':'0'},children =[
                dcc.Link('Inicio', href='/', style={'color': 'white', 'fontSize': '15px','width': '80%','marginBottom':'8px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Serie de tiempo', href='/time-series', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),
                dcc.Link('Modelo Predictivo', href='/predictive-model', style={'color': 'white','width': '80%','marginBottom':'8px',  'fontSize': '15px', 'textDecoration': 'none', 'padding': '10px', 'borderRadius': '5px', 'backgroundColor': '#ffa888', 'textAlign': 'center'}),

            ]),
            html.Div(id='explain-content')
        ]),

        html.Section(id='page-content', className='section',style={'gridArea':'section', 'gridColumnStart':'2','gridColumnEnd':'4','gridRowStart':'1','gridRowEnd':'5','backgroundColor': 'white','border':'1px solid #cbc4ad', 'borderRadius':'5px', 'padding': '20px'}),
    ]),
])

@app.callback(Output('explain-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return html.Div() 
    elif pathname == '/predictive-model':
        return ()
    else:
        return (
            )
      
# Update the index
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/time-series':
        return (html.Div([
                 html.H2('Serie de tiempo - Anomalía de Temperatura Global (1850 - 2023)'),
                dcc.Graph(id='timeseries-graph'),
                ]),
                html.Div([
                    html.H2(" Boxplot - Variación mensual multianual (1850 - 2023)",),
                    dcc.Graph(id='boxplot')]),
                html.Div([
                    html.Div([
                        html.H2("Heatmap- Exploración de patrones y tendencias"),
                        dcc.Graph(id='heatmap'),]),
                    html.Div([
                        html.H1("Barplot - Promedios Decenales de Anomalías de Temperatura"),
                        dcc.Graph(id='bar-chart'),
                    ])
                ]),
                html.Div([
                    dcc.Dropdown(
                        id='year-picker',
                        options=[{'label': i, 'value': i} for i in df_tseries['year'].unique()],
                        value=[df_tseries['year'].max()],
                        multi=True
                    ),
                     dcc.Graph(id='temperature-graph')
                ])
            )
    elif pathname == '/predictive-model':
        return html.Div([
            html.H3('Estás en la página del modelo predictivo')
        ])
    else:
        return (
            html.H4('Sobre el conjunto de datos',style={'fontSize':'1rem', 'textAlign':'left'}),
            html.P('Presentamos el análisis de un conjunto de datos llamado HadCRUT5, que mide los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. HadCRUT5 presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra.', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Para revisar a detalle el proceso de obtención y procesamiento de los datos, visite la información de referencia. ', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
            html.P('Morice, C. P., Kennedy, J. J., Rayner, N. A., Winn, J. P., Hogan, E., Killick, R. E., Dunn, R. J. H., Osborn, T. J., Jones, P. D., & Simpson, I. R. (2021). An Updated Assessment of Near-Surface Temperature Change From 1850: The HadCRUT5 Data Set. Journal of Geophysical Research: Atmospheres, 126(3), e2019JD032361. https://doi.org/10.1029/2019JD032361', style={'fontSize':'0.8rem', 'textAlign':'left','lineHeight':'150%'}),
        )


# Definir la función callback
@app.callback(Output('timeseries-graph', 'figure'),
              [Input('timeseries-graph', 'hoverData')])
def update_graph(hoverData):
    # Calcular la media móvil y el intervalo de confianza
    df_timeseries = calculate_rolling_mean(df_tseries.copy())  # Asegúrate de tener una copia del DataFrame original
    trace_data = []

    # Agregar la serie temporal original
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['value'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Original Data',
        'line': {'color': 'blue'}
    })

    # Agregar la media móvil
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['rolling_mean'],
        'type': 'line',
        'mode': 'lines',
        'name': '12-Month Rolling Mean',
        'line': {'color': 'red'}
    })

    # Agregar el intervalo de confianza
    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['upper_band'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Upper Band',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    trace_data.append({
        'x': df_timeseries['date'],
        'y': df_timeseries['lower_band'],
        'type': 'line',
        'mode': 'lines',
        'name': 'Lower Band',
        'line': {'color': 'green', 'dash': 'dash'}
    })

    # Marcar la media correspondiente al año cuando se pasa el ratón sobre la gráfica
    if hoverData:
        year_hovered = hoverData['points'][0]['x'].split('-')[0]
        mean_value = df_timeseries[df_timeseries['year'] == int(year_hovered)]['rolling_mean'].iloc[0]
        trace_data.append({
            'x': [hoverData['points'][0]['x']],
            'y': [mean_value],
            'type': 'scatter',
            'mode': 'markers',
            'name': f'Mean {year_hovered}',
            'marker': {'color': 'black', 'size': 10},
            'text': [f'Mean {year_hovered}']
        })

    figure = {
        'data': trace_data,
        'layout': {
            'xaxis': {'title': 'Fecha'},
            'yaxis': {'title': 'Valor'},
            'margin': {'b': 30, 'r': 10, 'l': 60, 't': 50},
            'legend': {'x': 0, 'y': 1}
        }
    }

    return figure

@app.callback(Output('boxplot', 'figure'),
              [Input('boxplot', 'hoverData')])
def update_boxplot(hoverData):
    # Crear el boxplot con Plotly Graph Objects
    fig = go.Figure()

    # Agregar boxplots para cada mes
    for month in df_tseries['month'].unique():
        data = df_tseries[df_tseries['month'] == month]
        fig.add_trace(go.Box(x=data['month'], y=data['value'], name=str(month), width=0.5))

    # Agregar la línea de tendencia (media multianual)
    trend_line = df_tseries.groupby('month')['value'].mean().reset_index()
    
    # Ordenar los datos por el orden de los meses
    ordered_months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    trend_line['month'] = pd.Categorical(trend_line['month'], categories=ordered_months, ordered=True)
    trend_line = trend_line.sort_values('month')

    fig.add_trace(go.Scatter(x=trend_line['month'], y=trend_line['value'],
                             mode='lines', line=dict(color='black', dash='dash'),
                             name='Media Multianual'))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Mes'}, yaxis={'title': 'Anomalía de Temperatura (°C)'},
                      boxmode='group')  # Para agrupar los boxplots en el eje x

    return fig

@app.callback(Output('heatmap', 'figure'),
              [Input('heatmap', 'hoverData')])
def update_heatmap(hoverData):
    # Crear el mapa de calor con Plotly Graph Objects
    fig = go.Figure()

    # Añadir el mapa de calor
    fig.add_trace(go.Heatmap(x=heatmap_data.columns, y=heatmap_data.index, z=heatmap_data.values,
                             colorscale='RdBu', colorbar=dict(title='Anomalía de Temperatura'), reversescale=True))

    # Ajustar el diseño del gráfico
    fig.update_layout(xaxis={'title': 'Año'}, yaxis={'title': 'Mes', 'categoryorder': 'array', 'categoryarray': month_order})

    return fig


@app.callback(Output('bar-chart', 'figure'),
              [Input('bar-chart', 'hoverData')])
def update_bar_chart(hoverData):
    # Calcular los promedios decenales
    decade_means = df_tseries.groupby('decade').agg({'value': 'mean', 'color': 'first'}).reset_index()

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(decade_means, x='decade', y='value', color='color',
                 labels={'value': 'Anomalía de Temperatura (°C)'}, 
                 color_discrete_map={'red': 'red', 'blue': 'blue'})

    # Modificar las etiquetas de los ticks en el eje x
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=decade_means['decade'],
            ticktext=[f"{decade}s" for decade in decade_means['decade']]
        ),
        showlegend=False
    )

    return fig

@app.callback(
    Output('temperature-graph', 'figure'),
    [Input('year-picker', 'value')]
)
def update_figure(selected_years):
    traces = []
    markers = ['circle', 'square', 'diamond', 'cross', 'x']

    # Añade las líneas para los años más cálidos
    for i, year in enumerate(top_years):
        df_year = df_tseries[df_tseries['year'] == year]
        traces.append(go.Scatter(
            x=df_year['month'],
            y=df_year['value'],
            mode='lines+markers',
            name=str(year),
            marker=dict(
                symbol=markers[i % len(markers)],
                size=10
            )
        ))

    # Añade las líneas para los años seleccionados
    for i, selected_year in enumerate(selected_years):
        df_selected = df_tseries[df_tseries['year'] == selected_year]
        traces.append(go.Scatter(
            x=df_selected['month'],
            y=df_selected['value'],
            mode='lines+markers',
            name=str(selected_year),
            line={'color': 'black', 'width': 2},
            marker=dict(
                symbol=markers[(i + len(top_years)) % len(markers)],
                size=10
            )
        ))

    # Añade la línea para los valores máximos
    traces.append(go.Scatter(
        x=np.arange(1, 13),
        y=max_values,
        mode='lines',
        name='Máximo histórico (1850 - 2022)',
        line={'color': 'darkgrey', 'width': 2.5}
    ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Mes', 'tickvals': list(range(1, 13)), 'ticktext': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']},
            yaxis={'title': 'Anomalía de temperatura (°C)'},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    #app.run_server(debug=True) # <- For testing purposes
    app.run_server(debug=True, host='0.0.0.0', port=9000) # <- To Dockerize the Dash
    

Este código inicia de forma sencilla con el cargue de los datos. 
Seguido, el código crea una función llamada calculate_rolling_mean que trabaja con los datos para calcular la media móvil y establecer límites de confianza. Luego, basándose en estos cálculos, la función añade nuevas características al conjunto de datos: una para representar la media de esos valores y dos más para mostrar un rango de valores que son más o menos estándar, todo esto usando la idea de "desviación estándar" y un número específico que indica cuánto extender esos límites (por defecto, 1.96). Al final, devuelve el conjunto de datos actualizado, ahora con estos números adicionales que muestran cómo los valores varían a lo largo del tiempo.


Luego, se realiza una transformación de los datos almacenados en el DataFrame df_tseries para generar un mapa de calor (heatmap_data). Esta transformación reorganiza los datos para representar los años en el eje horizontal (X) y los meses en el eje vertical (Y). Despues, crea una nueva columna llamada 'decade' que agrupa los años por década para un análisis más general. Además, introduce una columna llamada 'color', que asigna un color ('rojo' o 'azul') basado en la condición de anomalía de los valores. Posteriormente, se generan columnas adicionales para el año y el mes, lo que permite un fácil acceso y manipulación de estos datos temporales.

También se identifica los tres años más cálidos almacenándolos en la variable top_years. También encuentra los valores máximos de temperatura para cada mes, guardándolos en la variable max_values. 


Seguido, se establece la estructura de la interfaz de usuario de una aplicación Dash, dividida en dos secciones: una barra de navegación y un área de contenido. La barra de navegación tiene enlaces para dirigirse a diferentes secciones de la aplicación. Dos funciones de callback se encargan de cambiar el contenido de la página según la URL seleccionada, mostrando gráficos y texto relevante. Además, se incluye información descriptiva sobre el conjunto de datos HadCRUT5 para contextualizar el análisis de la temperatura global. Este código permite una experiencia de usuario interactiva y dinámica.

Posteriormente, se presenta una función de callback diseñada para actualizar un gráfico en una aplicación Dash cuando se detecta un evento de "hover" (pasar el ratón por encima) en dicho gráfico. Al activarse el evento de "hoverData", la función realiza varias acciones: primero, calcula la media móvil y los límites del intervalo de confianza utilizando la función calculate_rolling_mean. Luego, crea una lista llamada trace_data para almacenar los datos del gráfico, incluyendo la serie temporal original, la media móvil, y los límites superior e inferior del intervalo de confianza, cada uno representado por líneas de colores distintos. Además, cuando se pasa el ratón sobre el gráfico, la función identifica el año correspondiente al punto en el que está posicionado el ratón y marca la media de dicho año en el gráfico con un punto negro.

Siguiendo la misma linea, tenemos dos bloques de código que presentan funciones de callback. La primera función, update_boxplot, genera un gráfico de cajas (boxplot) utilizando Plotly Graph Objects. Esta función itera sobre cada mes en el conjunto de datos y agrega un boxplot correspondiente a cada mes, representando la distribución de las anomalías de temperatura en ese mes específico. Además, añade una línea de tendencia que muestra la media multianual de las anomalías de temperatura para cada mes. El diseño del gráfico se ajusta para mostrar en el eje x los meses y en el eje y las anomalías de temperatura en grados Celsius.

La segunda función, update_heatmap, crea un gráfico de mapa de calor con Plotly Graph Objects. Este mapa de calor representa las anomalías de temperatura a lo largo de los años y meses. Se utiliza un heatmap para visualizar las variaciones de temperatura a lo largo del tiempo. La función agrega el heatmap utilizando los datos ya preparados en la variable heatmap_data y ajusta el diseño del gráfico para mostrar los años en el eje x y los meses en el eje y, manteniendo el orden específico de los meses.

Los últimos bloques de código son : update_bar_chart, genera un gráfico de barras que muestra los promedios decenales de las anomalías de temperatura. Calcula estos promedios agrupando los datos por década y luego utiliza Plotly Express para crear el gráfico de barras, donde cada barra representa el promedio de temperatura de una década particular. Modifica las etiquetas en el eje x para mostrar décadas y colores que representan las tendencias de temperaturas más cálidas o más frías.

La segunda función, update_figure, actualiza un gráfico de líneas dinámico. Esta función utiliza los años seleccionados para representar las anomalías de temperatura a lo largo de los meses. Genera líneas para los años más cálidos, los años seleccionados y también agrega una línea que muestra los valores máximos históricos de anomalías de temperatura para cada mes. Ajusta el diseño del gráfico para mostrar los meses en el eje x y las anomalías de temperatura en grados Celsius en el eje y, permitiendo que el usuario explore las variaciones de temperatura a lo largo de los años.

La condición if __name__ == '__main__': permite ejecutar la aplicación Dash, ya sea en modo de depuración (debug=True) para pruebas o configurado para ser utilizado en un entorno Docker (host='0.0.0.0', port=9000). Estas funciones de callback son fundamentales para actualizar y renderizar gráficos dinámicos en función de la interacción del usuario en la aplicación Dash.